In [52]:
import os
import cv2
import random
import rasterio
import numpy as np
import geopandas as gpd
from rasterio.mask import mask
from rasterio.plot import show
import matplotlib.pyplot as plt
from shapely.geometry import box
from torchvision import transforms
from PIL import Image, ImageEnhance
from rasterio.windows import Window

In [ ]:
def save_raster(output_raster, data, profile):
    """
    Save raster data while keeping metadata.
    """
    with rasterio.open(output_raster, "w", **profile) as dest:
        for i in range(data.shape[0]):
            dest.write(data[i], i + 1)
            
def rotateAndDeleteEmptySpace(fileName, outputName, degreeOfRotation=24):
    with rasterio.open(fileName) as src:
        data = src.read()  # shape: (bands, height, width)
        profile = src.profile

    # Rotate the entire image band-by-band, and track global crop box
    rotated_bands = []
    masks = []

    for i in range(data.shape[0]):
        pil_img = Image.fromarray(data[i])
        rotated = pil_img.rotate(degreeOfRotation, expand=True)
        rotated_np = np.array(rotated)
        rotated_bands.append(rotated_np)

        # Create mask to find non-zero regions
        masks.append(rotated_np > 0)

    # Combine masks to find common cropping area
    combined_mask = np.any(np.stack(masks), axis=0)
    non_empty_rows = np.any(combined_mask, axis=1)
    non_empty_cols = np.any(combined_mask, axis=0)
    
    min_row, max_row = np.where(non_empty_rows)[0][[0, -1]]
    min_col, max_col = np.where(non_empty_cols)[0][[0, -1]]

    # Crop all rotated bands to the same bounding box
    cropped_bands = [
        band[min_row:max_row + 1, min_col:max_col + 1]
        for band in rotated_bands
    ]

    cropped_data = np.stack(cropped_bands)

    # Update profile
    profile.update(width=cropped_data.shape[2], height=cropped_data.shape[1])

    save_raster(outputName, cropped_data, profile)

def loopRemoveFile(path):
    for dayFolder in os.listdir(path):
        dayFolderPath = path + "/" + dayFolder
        for eachClippedFolder in os.listdir(dayFolderPath):
            eachClippedFolderPath = dayFolderPath + "/" + eachClippedFolder
            for file in os.listdir(eachClippedFolderPath):
                print(file[-15:])
                if(file[-8:] == ".aux.xml"):
                    if file.startswith("._"):
                        continue
                    junkFile = os.path.join(eachClippedFolderPath, file)
                    os.remove(junkFile)
                    print(junkFile, " deleted.")

def loopAdjustOriginalTifFileOrientation(mainPath):
    for dayFolder in os.listdir(mainPath):
        dayFolderPath = os.path.join(mainPath, dayFolder)
        for eachClippedFolder in os.listdir(dayFolderPath):
            eachClippedFolderPath = os.path.join(dayFolderPath, eachClippedFolder)
            for file in os.listdir(eachClippedFolderPath):
                if(file[-12:] == "original.tif"):
                    if file.startswith("._"):
                        continue

                    inputPath = os.path.join(eachClippedFolderPath, file)
                    newFileName = file[:-12] + "originalFixTilt.tif"
                    outputPath = os.path.join(eachClippedFolderPath, newFileName)

                    if os.path.exists(outputPath):
                        print("File already exists: ", outputPath)
                    else:
                        rotateAndDeleteEmptySpace(inputPath, outputPath, 24)
                        print("saved rotated tif file to: ", outputPath)

In [54]:
loopRemoveFile("/Volumes/PortableSSD/dataForProcess/2025MainData/RGB")
# loopAdjustOriginalTifFileOrientation("/Volumes/PortableSSD/dataForProcess/2025MainData/RGB")

_1_original.tif
inalFixTilt.tif
_1_original.jpg
_1_original.jpg
_2_original.tif
inalFixTilt.tif
_2_original.jpg
_3_original.tif
inalFixTilt.tif
_3_original.jpg
_4_original.tif
inalFixTilt.tif
_4_original.jpg
_5_original.tif
inalFixTilt.tif
_5_original.jpg
_6_original.tif
inalFixTilt.tif
_6_original.jpg
_7_original.tif
inalFixTilt.tif
_7_original.jpg
_8_original.tif
inalFixTilt.tif
_8_original.jpg
_9_original.tif
inalFixTilt.tif
_9_original.jpg
10_original.tif
inalFixTilt.tif
10_original.jpg
11_original.tif
inalFixTilt.tif
11_original.jpg
12_original.tif
inalFixTilt.tif
12_original.jpg
13_original.tif
inalFixTilt.tif
13_original.jpg
14_original.tif
inalFixTilt.tif
14_original.jpg
15_original.tif
inalFixTilt.tif
15_original.jpg
16_original.tif
inalFixTilt.tif
16_original.jpg
17_original.tif
inalFixTilt.tif
17_original.jpg
18_original.tif
inalFixTilt.tif
18_original.jpg
19_original.tif
inalFixTilt.tif
19_original.jpg
20_original.tif
inalFixTilt.tif
20_original.jpg
21_original.tif
inalFixT

In [55]:
# rotateAndDeleteEmptySpace("/Volumes/PortableSSD/dataForProcess/2025MainData/RGB/RGB_202503010913/RGB_202503010913_1/RGB_202503010913_1_original.tif", "./test.tif", 23)